# Import Libraries

In [1]:
!pip install pandas

     --------------------------------------- 10.0/10.0 MB 50.7 kB/s eta 0:00:00
     ------------------------------------- 505.5/505.5 kB 65.9 kB/s eta 0:00:00


In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import pandas as pd
from collections import deque
#from moviepy.editor import *

In [2]:
# Download Model uncomment if running in Google Colab or Do not have the trained model
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vNp1c7ouaWtaHb5GdLZX6OV5ldpXwg_Z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vNp1c7ouaWtaHb5GdLZX6OV5ldpXwg_Z" -O Suspicious_Human_Activity_Detection_VGG16_LSTM_Model.h5 && rm -rf /tmp/cookies.txt

# Declare Constants

In [2]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
 
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 30
 
# Specify the directory containing the UCF50 dataset. 
DATASET_DIR = "Dataset"
 
# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
# CLASSES_LIST = ["walking", "fight",  "shoplifting", "running"]
CLASSES_LIST = ["handclapping", "jogging", "stealing", "lifting"]

# Load Model

In [3]:
model = load_model('Suspicious_Human_Activity_Detection_LRCN_Model_20240603.h5')

# Prediction of Single Action

In [4]:
def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    '''
    This function will perform single action recognition prediction on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a list to store video frames we will extract.
    frames_list = []
    
    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read a frame.
        success, frame = video_reader.read() 

        # Check if frame is not read properly then break the loop.
        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255
        
        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = model.predict(np.expand_dims(frames_list, axis = 0))[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]
    
    # Display the predicted action along with the prediction confidence.
    print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')
        
    # Release the VideoCapture object. 
    video_reader.release()

In [4]:
predict_single_action("Predict/fight.avi",SEQUENCE_LENGTH)

NameError: name 'model' is not defined

In [5]:
predict_single_action("Dataset/running/person02_running_d4_uncomp.avi",SEQUENCE_LENGTH)

Action Predicted: jogging
Confidence: 0.9953427314758301


In [6]:
predict_single_action("Dataset/walking/person03_walking_d3_uncomp.avi",SEQUENCE_LENGTH)

Action Predicted: jogging
Confidence: 0.9962804913520813


In [7]:
predict_single_action("Dataset/shoplifting/Shoplifting021_x264.mp4",SEQUENCE_LENGTH)

Action Predicted: stealing
Confidence: 0.6817734837532043


In [8]:
predict_single_action("Dataset/shoplifting/Shoplifting054_x264.mp4",SEQUENCE_LENGTH)

Action Predicted: stealing
Confidence: 0.5286037921905518


In [13]:
predict_single_action("Predict/walking.avi",SEQUENCE_LENGTH)

ValueError: in user code:

    C:\Users\Phi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\Phi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Phi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Phi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Phi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Phi\AppData

# All Action Prediction in Video

In [9]:
def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH):
    '''
    This function will perform action recognition on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    output_file_path: The path where the ouput video with the predicted action being performed overlayed will be stored.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize the VideoWriter Object to store the output video in the disk.
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc(*'DIVX'), 
                                   video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))

    # Declare a queue to store video frames.
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)

    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Iterate until the video is accessed successfully.
    while video_reader.isOpened():

        # Read the frame.
        ok, frame = video_reader.read() 
        
        # Check if frame is not read properly then break the loop.
        if not ok:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list.
        frames_queue.append(normalized_frame)

        # Check if the number of frames in the queue are equal to the fixed sequence length.
        if len(frames_queue) == SEQUENCE_LENGTH:

            # Pass the normalized frames to the model and get the predicted probabilities.
            predicted_labels_probabilities = model.predict(np.expand_dims(frames_queue, axis = 0))[0]

            # Get the index of class with highest probability.
            predicted_label = np.argmax(predicted_labels_probabilities)

            # Get the class name using the retrieved index.
            predicted_class_name = CLASSES_LIST[predicted_label]

        # Write predicted class name on top of the frame.
        cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write The frame into the disk using the VideoWriter Object.
        video_writer.write(frame)
        
    # Release the VideoCapture and VideoWriter objects.
    video_reader.release()
    video_writer.release()

In [10]:
%pwd

'C:\\Users\\Phi\\Desktop\\suspicious-detection-phimhn\\Suspicious-Human-Activity-Detection-LRCN'

In [16]:
%%time
predict_on_video("Predict/Human-Activity.avi","Human-Activity-Prediction.avi",SEQUENCE_LENGTH)

Wall time: 557 ms


In [11]:
%%time
predict_on_video("Predict/hanoi_stealing.mp4","output/hanoi_stealing2.mp4",SEQUENCE_LENGTH)

Wall time: 2min 7s


In [12]:
%%time
predict_on_video("Predict/person23_walking_d4_uncomp.avi","output/person23_walking_d4_uncomp.avi",SEQUENCE_LENGTH)

Wall time: 17.6 s


In [14]:
%%time
predict_on_video("Predict/moctui01.mp4","output/moctui01.mp4",SEQUENCE_LENGTH)

Wall time: 58.4 s


In [13]:
%%time
predict_on_video("Predict/tromsieuthi01.mp4","output/tromsieuthi01.mp4",SEQUENCE_LENGTH)

Wall time: 35.8 s


In [14]:
%%time
predict_on_video("Predict/Stealing006_x264.mp4","output/Stealing006_x264.mp4",SEQUENCE_LENGTH)

Wall time: 403 ms


In [15]:
%%time
predict_on_video("Predict/layxe01.mp4","output/layxe01.mp4",SEQUENCE_LENGTH)

Wall time: 21.5 s


In [16]:
%%time
predict_on_video("Predict/laydo01.mp4","output/laydo01.mp4",SEQUENCE_LENGTH)

Wall time: 18.8 s


In [17]:
%%time
predict_on_video("Predict/tromxemay.mp4","output/tromxemay.mp4",SEQUENCE_LENGTH)

Wall time: 281 ms


In [23]:
%%time
predict_on_video("Predict/tromxemay1.mp4","output/tromxemay1.mp4",SEQUENCE_LENGTH)

Wall time: 435 ms


In [24]:
%%time
predict_on_video("Predict/binhtantromxemay.mp4","output/binhtantromxemay.mp4",SEQUENCE_LENGTH)

Wall time: 15.5 s


In [25]:
%%time
predict_on_video("Predict/binhchanhdienthoai.mp4","output/binhchanhdienthoai.mp4",SEQUENCE_LENGTH)

Wall time: 51.1 s


In [26]:
%%time
predict_on_video("Predict/phakepsat.mp4","output/phakepsat.mp4",SEQUENCE_LENGTH)

Wall time: 1min 7s


In [19]:
%%time
predict_on_video("Dataset/running/person02_running_d4_uncomp.avi","output/person02_running_d4_uncomp.avi",SEQUENCE_LENGTH)

Wall time: 13 s


In [20]:
%%time
predict_on_video("Dataset/running/person02_running_d4_uncomp.avi","output/person02_running_d4_uncomp.avi",SEQUENCE_LENGTH)

Wall time: 14.8 s


In [20]:
!pip install moviepy --upgrade
!pip install ffmpeg --upgrade
!pip install --upgrade decorator==4.4.2

In [21]:
VideoFileClip("Human-Activity-Prediction.avi", audio=False).ipython_display()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



TypeError: must be real number, not NoneType